In [ ]:
import numpy as np
import glob
from PIL import Image
from numpy import expand_dims
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import shutil
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import load_model
from sklearn.metrics.pairwise import cosine_similarity
import scipy
from scipy.stats import shapiro, levene, ttest_ind, mannwhitneyu
import tensorflow as tf
import pandas as pd

In [ ]:
# validation set을 최종 모델에 통화 시킨 feature vectors
validation_matrix = np.load("/content/drive/Shared drives/선빵팀 :) 2/validation_matrix/validation_matrix_woong.npy")


# 정규성, 동분산성, 평균검정 함수

In [ ]:
# 정규성 검정
def normal(group):
    result = shapiro(group)
    w = result[0]
    p = result[1]
    if p <0.05:
        normality = 1
    else:
        normality = 0
    return normality, p

# 동분산성 검정
def homogenity(group1, group2):
    w, p = levene(group1, group2)
    if p < 0.05:
        homo = 1
    else:
        homo = 0
    return homo, p

# 두 그룹 간 평균 t 검정 (정규성 만족할 경우)
def t_test(group1, group2, homo):
    homo = (homo==1)  
    w, p = ttest_ind(group1, group2, equal_var = homo)
    #w, p = w[0], p[0]
    if p < 0.05:
        diff = 1
    else:
        diff = 0
    return diff, p


# 두 그룹 간 평균 검정 - Mann-whitney test (정규성 불만족할 경우)
def man_whitney(group1, group2, home):
    w, p = scipy.stats.mannwhitneyu(group1, group2, use_continuity=True, alternative=None)
    if p < 0.05:
        diff = 1
    else:
        diff = 0
    return diff, p

# Inner_similarity들 끼리 평균 차이

## 카테고리 별 내부 유사도 집단 정리하기(inner_cosine_similarity)

In [ ]:
def inner_cosine_similarity(validation_matrix):
    similarity_matrix = cosine_similarity(validation_matrix)

    category_cosine_sim = {}

    key = 0 # 딕셔너리 key (카테고리)
    for i in range(0,91):
        first = 111*i
        last = 111*(i+1)
        # i 번째 카테고리
        similarity_matrix_i = similarity_matrix[first:last,first:last]


        value_list = []
        for row in range(0, 111):
            for col in range(row+1, 111):
                value = similarity_matrix_i[row][col]
                value_list.append(value)
        category_cosine_sim[str(key)] = np.array(value_list)
        key = key + 1
    return category_cosine_sim

In [ ]:
inner_cosine_sim = inner_cosine_similarity(validation_matrix)
inner_cosine_sim

In [ ]:
def inner_cosine_means(inner_cosine_sim):
    category_cosine_means = {}

    for key, value in inner_cosine_sim.items():
        category_cosine_means[key] = np.mean(value)

    return category_cosine_means

## 카테고리 내부 유사도 평균

In [ ]:
inner_consine_means = inner_cosine_means(inner_cosine_sim)
inner_consine_means

{'0': 0.8804551,
 '1': 0.79425526,
 '10': 0.8931132,
 '11': 0.83618253,
 '12': 0.8626297,
 '13': 0.8588592,
 '14': 0.81243634,
 '15': 0.8598615,
 '16': 0.8684837,
 '17': 0.8546861,
 '18': 0.86800337,
 '19': 0.9325699,
 '2': 0.7979257,
 '20': 0.88121784,
 '21': 0.9068284,
 '22': 0.8738793,
 '23': 0.92245334,
 '24': 0.89520407,
 '25': 0.8679233,
 '26': 0.9055387,
 '27': 0.85302216,
 '28': 0.8722421,
 '29': 0.7616308,
 '3': 0.85359824,
 '30': 0.87761414,
 '31': 0.7187521,
 '32': 0.78913707,
 '33': 0.8247333,
 '34': 0.91448194,
 '35': 0.75482196,
 '36': 0.79624707,
 '37': 0.8492677,
 '38': 0.87375104,
 '39': 0.7701811,
 '4': 0.7924588,
 '40': 0.8306413,
 '41': 0.8267289,
 '42': 0.8538949,
 '43': 0.8380264,
 '44': 0.80225945,
 '45': 0.7547371,
 '46': 0.78742915,
 '47': 0.81521183,
 '48': 0.7976857,
 '49': 0.8735412,
 '5': 0.93255174,
 '50': 0.8191592,
 '51': 0.9002868,
 '52': 0.86935407,
 '53': 0.8448107,
 '54': 0.85331935,
 '55': 0.84722996,
 '56': 0.7428603,
 '57': 0.71241856,
 '58': 0.73

## 모든 카테고리의 정규성 검정하기

In [ ]:
category_nomality = []
for category, cosine_sims in inner_cosine_sim.items():
    normality, p = normal(cosine_sims)
    category_nomality.append(normality)
category_nomality = np.array(category_nomality)

/usr/local/lib/python3.6/dist-packages/scipy/stats/morestats.py:1676: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


In [ ]:
normal_category = np.where(category_nomality == 1)
normal_category

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
        68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
        85, 86, 87, 88, 89, 90]),)

In [ ]:
non_normal_category = np.where(category_nomality == 0)
non_normal_category

(array([], dtype=int64),)

## 카테고리 간 평균 차이 검정

In [ ]:
category = np.array(range(0,91))
category

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90])

In [ ]:
# 두 그룹 간 평균 t 검정 (정규성 만족할 경우)
category = np.array(range(0,91))

similarity_matrix = cosine_similarity(validation_matrix)

mean_test = {} # I그룹 내부 코사인 유사도 vs i그룹 - j그룹 코사인 유사도 평균 검정 
standard_category = 1
for i in category:
    for j in category[standard_category:]:
        # 동분산 테스트
        group_i = similarity_matrix[111*i:111*(i+1),111*i:111*(i+1)].reshape((12321,)) # 1 ㅇ
        group_j = similarity_matrix[111*i:111*(i+1),111*j:111*(j+1)].reshape((12321,))
        homo, p = homogenity(group_i, group_j)

        compare_group_label = str(i) + "_" + str(j)
        mean_test[compare_group_label] = [t_test(group_i, group_j, homo)]

    standard_category += 1

In [ ]:
len(list(mean_test.keys()))

4095

In [ ]:
# 임의의 두 카테고리 간 평균 차이가 없는 경우 검사
for key, value in mean_test.items():
    d = value[0][0]
    p = value[0][1]
    if d == 0:
        print(key, "are significantly same")
print("done")

done


# Inner vs Outer similarity 평균 차이

## 각 집단 정리하기

In [ ]:
validation_matrix.shape

(10101, 2048)

In [ ]:
# 코사인 유사도
similarity_matrix = cosine_similarity(validation_matrix)

In [ ]:
similarity_matrix.shape

(10101, 10101)

In [ ]:
category

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90])

In [ ]:
# 그룹 내외 유사도

category = category
inner_outer = {}

for i in category:
    key = str(i)
    
    first = 111*i
    last = 111*(i+1)
    # i 번째 카테고리
    similarity_matrix_i = similarity_matrix[first:last,first:last]

    inner_value = []
    for row in range(0, 111):
        for col in range(row+1, 111):
            value = similarity_matrix[row][col]
            inner_value.append(value)
    inner_value = np.array(inner_value)

    outer_value = np.concatenate((similarity_matrix[i*111:111*(i+1),:i*111], similarity_matrix[i*111:111*(i+1), (i+1)*111:]), axis = 1)
    outer_value = outer_value.reshape(1108890,)
    value = [inner_value, outer_value]
    inner_outer[key] = value

In [ ]:
len(inner_outer["0"][1])

1108890

In [ ]:
inner_outer_means = {}
for key, values in inner_outer.items():
    inner_values = values[0]
    outer_values = values[1]

    inner_values_mean = round(np.mean(inner_values), 4)
    outer_values_mean = round(np.mean(outer_values), 4)

    inner_outer[key] = [inner_values_mean, outer_values_mean]

In [ ]:
inner_outer_means

{'0': [0.9863, 0.9747],
 '1': [0.9642, 0.9658],
 '10': [0.9954, 0.9628],
 '11': [0.9815, 0.9731],
 '12': [0.9868, 0.9791],
 '13': [0.9894, 0.9792],
 '14': [0.963, 0.9655],
 '15': [0.9871, 0.9777],
 '16': [0.9872, 0.9788],
 '17': [0.9798, 0.9755],
 '18': [0.9831, 0.9773],
 '19': [0.9863, 0.9757],
 '2': [0.9797, 0.9746],
 '20': [0.9835, 0.9633],
 '21': [0.9871, 0.9665],
 '22': [0.9859, 0.9787],
 '23': [0.9852, 0.9714],
 '24': [0.9845, 0.9757],
 '25': [0.9785, 0.9679],
 '26': [0.9888, 0.9728],
 '27': [0.9865, 0.9784],
 '28': [0.9831, 0.9742],
 '29': [0.9815, 0.9763],
 '3': [0.9792, 0.9753],
 '30': [0.9877, 0.9759],
 '31': [0.9774, 0.9721],
 '32': [0.9835, 0.9772],
 '33': [0.9798, 0.9758],
 '34': [0.9879, 0.9789],
 '35': [0.9786, 0.9749],
 '36': [0.9771, 0.9743],
 '37': [0.9833, 0.9758],
 '38': [0.9882, 0.9794],
 '39': [0.9795, 0.9707],
 '4': [0.9827, 0.9773],
 '40': [0.9842, 0.9756],
 '41': [0.9813, 0.9747],
 '42': [0.9863, 0.9788],
 '43': [0.9805, 0.9749],
 '44': [0.9804, 0.9729],
 '45':

In [ ]:
df = pd.DataFrame(inner_outer_means_group, ("inner_similarity","outer_similarity"))
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90
inner_similarity,0.9863,0.9642,0.9797,0.9792,0.9827,0.9858,0.9758,0.9833,0.9635,0.9843,0.9954,0.9815,0.9868,0.9894,0.9630,0.9871,0.9872,0.9798,0.9831,0.9863,0.9835,0.9871,0.9859,0.9852,0.9845,0.9785,0.9888,0.9865,0.9831,0.9815,0.9877,0.9774,0.9835,0.9798,0.9879,0.9786,0.9771,0.9833,0.9882,0.9795,...,0.9908,0.9821,0.9828,0.9844,0.9812,0.9744,0.9790,0.9797,0.9836,0.9889,0.9783,0.9890,0.9671,0.9873,0.9913,0.9682,0.9793,0.9765,0.9736,0.9832,0.9806,0.9814,0.9707,0.9709,0.9664,0.9579,0.9867,0.9681,0.9833,0.9925,0.9792,0.9867,0.9755,0.9790,0.9841,0.9835,0.9859,0.9665,0.975,0.9942
outer_similarity,0.9747,0.9658,0.9746,0.9753,0.9773,0.9771,0.9739,0.9769,0.9617,0.9738,0.9628,0.9731,0.9791,0.9792,0.9655,0.9777,0.9788,0.9755,0.9773,0.9757,0.9633,0.9665,0.9787,0.9714,0.9757,0.9679,0.9728,0.9784,0.9742,0.9763,0.9759,0.9721,0.9772,0.9758,0.9789,0.9749,0.9743,0.9758,0.9794,0.9707,...,0.9809,0.9646,0.9766,0.9725,0.9767,0.9694,0.9717,0.9742,0.9702,0.9782,0.9744,0.9767,0.9690,0.9775,0.9761,0.9664,0.9731,0.9745,0.9720,0.9750,0.9762,0.9753,0.9601,0.9657,0.9667,0.9612,0.9766,0.9686,0.9665,0.9769,0.9735,0.9727,0.9699,0.9742,0.9564,0.9762,0.9770,0.9660,0.970,0.9762


In [ ]:
df.to_csv('/content/drive/My Drive/Colab Notebooks/inner_outer_similarity_resnet.csv')

In [ ]:
values = inner_outer_group["0"]
inner_group = values[0]
outer_group = values[1]
homo, p = homogenity(inner_group, outer_group)
homo = (homo == 1)

print("0번 카테고리", ttest_ind(inner_group, outer_group, equal_var = homo))

0번 카테고리 Ttest_indResult(statistic=430.66657535845127, pvalue=0.0)


In [ ]:
# 두 그룹 간 평균 t 검정 (정규성 만족할 경우)
mean_test = {} # I그룹 내부 코사인 유사도 vs i그룹 - j그룹 코사인 유사도 평균 검정 

i = 0
for values in inner_outer_group.values():
    # 동분산 테스트
    inner_group = values[0]
    outer_group = values[1]
    homo, p = homogenity(inner_group, outer_group)

    group_label = str(i)
    mean_test[group_label] = [t_test(inner_group, outer_group, homo)]
    
    i += 1

In [ ]:
mean_test

{'0': [(1, 0.0)],
 '1': [(1, 4.477146248425894e-07)],
 '10': [(1, 0.0)],
 '11': [(1, 6.954201063653806e-211)],
 '12': [(1, 0.0)],
 '13': [(1, 0.0)],
 '14': [(1, 2.488179414519797e-16)],
 '15': [(1, 0.0)],
 '16': [(1, 0.0)],
 '17': [(1, 3.303457563548069e-79)],
 '18': [(1, 9.150883786464795e-173)],
 '19': [(1, 0.0)],
 '2': [(1, 3.6177252262861795e-104)],
 '20': [(1, 0.0)],
 '21': [(1, 0.0)],
 '22': [(1, 1.722281569739976e-281)],
 '23': [(1, 0.0)],
 '24': [(1, 0.0)],
 '25': [(1, 0.0)],
 '26': [(1, 0.0)],
 '27': [(1, 0.0)],
 '28': [(1, 3.0161463012116865e-262)],
 '29': [(1, 7.055175925558563e-101)],
 '3': [(1, 5.738455979051018e-67)],
 '30': [(1, 0.0)],
 '31': [(1, 1.6516938617665788e-77)],
 '32': [(1, 2.2167116823485102e-173)],
 '33': [(1, 1.984885649789147e-73)],
 '34': [(1, 0.0)],
 '35': [(1, 7.595050409244989e-54)],
 '36': [(1, 3.0921003446251115e-28)],
 '37': [(1, 5.5900607841418196e-297)],
 '38': [(1, 0.0)],
 '39': [(1, 4.282707835060687e-192)],
 '4': [(1, 2.2757627501638744e-128)],

In [ ]:
len(list(mean_test.keys()))

91

In [ ]:
validation_matrix.shape

(10101, 2048)

In [ ]:
ith_category_matrix = {}
for i in range(0,91):
    start = i*111
    end = (i+1)*111
    ith_category_matrix[str(i)] = validation_matrix[start:end]

In [ ]:
ith_category_matrix # value들은 2차원

(111, 2048)

In [ ]:
i_j_cosine_similarity = {}
problem = []
for i in range(0,91):

    # 카테고리의 within similarity 계산
    within_similarity_matrix = cosine_similarity(ith_category_matrix[str(i)])
    within_similarity = []
    for n in range(0, 111):
        for m in range(n+1, 111):
            within_similarity.append(within_similarity_matrix[n][m])

    within_similarity = np.array(within_similarity)

    for j in range(i+1, 91): # i와 j 카테고리 선택



        # i와 j의 코사인 between similarity를 rank-1 array로 저장
        between_similarity_matrix = cosine_similarity(ith_category_matrix[str(i)], ith_category_matrix[str(i)])

        between_similarity = []
        for r in range(0, 111):
            for c in range(r, 111) : # 서로 다른 i,j 집단이기 때문에 유사도 행렬의 n,n 요소도 포함한다.
                between_similarity.append(between_similarity_matrix[r][c])

        between_similarity = np.array(between_similarity)

        # t 검정
        inner_group = within_similarity
        outer_group = between_similarity
        homo, p = homogenity(inner_group, outer_group)

        group_label = str(i)
        result = t_test(inner_group, outer_group, homo)
        
        if result[0] == 0:
            print(str(i) + "번째 카테고리와 " + str(j) + "번째 카테고리의 평균이 다르지 않다.")
            problem.append(str(i)+"_"+str(j))


39번째 카테고리와 40번째 카테고리의 평균이 다르지 않다.
39번째 카테고리와 41번째 카테고리의 평균이 다르지 않다.
39번째 카테고리와 42번째 카테고리의 평균이 다르지 않다.
39번째 카테고리와 43번째 카테고리의 평균이 다르지 않다.
39번째 카테고리와 44번째 카테고리의 평균이 다르지 않다.
39번째 카테고리와 45번째 카테고리의 평균이 다르지 않다.
39번째 카테고리와 46번째 카테고리의 평균이 다르지 않다.
39번째 카테고리와 47번째 카테고리의 평균이 다르지 않다.
39번째 카테고리와 48번째 카테고리의 평균이 다르지 않다.
39번째 카테고리와 49번째 카테고리의 평균이 다르지 않다.
39번째 카테고리와 50번째 카테고리의 평균이 다르지 않다.
39번째 카테고리와 51번째 카테고리의 평균이 다르지 않다.
39번째 카테고리와 52번째 카테고리의 평균이 다르지 않다.
39번째 카테고리와 53번째 카테고리의 평균이 다르지 않다.
39번째 카테고리와 54번째 카테고리의 평균이 다르지 않다.
39번째 카테고리와 55번째 카테고리의 평균이 다르지 않다.
39번째 카테고리와 56번째 카테고리의 평균이 다르지 않다.
39번째 카테고리와 57번째 카테고리의 평균이 다르지 않다.
39번째 카테고리와 58번째 카테고리의 평균이 다르지 않다.
39번째 카테고리와 59번째 카테고리의 평균이 다르지 않다.
39번째 카테고리와 60번째 카테고리의 평균이 다르지 않다.
39번째 카테고리와 61번째 카테고리의 평균이 다르지 않다.
39번째 카테고리와 62번째 카테고리의 평균이 다르지 않다.
39번째 카테고리와 63번째 카테고리의 평균이 다르지 않다.
39번째 카테고리와 64번째 카테고리의 평균이 다르지 않다.
39번째 카테고리와 65번째 카테고리의 평균이 다르지 않다.
39번째 카테고리와 66번째 카테고리의 평균이 다르지 않다.
39번째 카테고리와 67번째 카테고리의 평균이 다르지 않다.
39번째 카테고리와 68번째 카테고리의 평균이 다르지 않다.
39번째 카테고리와 69번

In [ ]:
problem

[]